In [436]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from matplotlib.pyplot import imshow
import numpy as np
import fnmatch
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pylab
import pandas as pd
%matplotlib inline
pylab.rcParams['figure.figsize'] = (10.0, 8.0)


In [78]:
dbname = 'coffee_detai1'
username = 'emclinden'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url

postgres://emclinden@localhost/coffee_detai1


In [79]:
sql_query = """
SELECT * FROM image_stats_del;
"""
data = pd.read_sql_query(sql_query,engine)

In [81]:
ntags_scale = (data.ntags - np.mean(data.ntags))/np.std(data.ntags)#,np.std(data.ntags)

In [88]:
bright_scale = (data.brightness - np.mean(data.brightness))/np.std(data.brightness)

In [91]:
comm_scale = (data.n_comments)

In [99]:
nc = data.n_comments.astype('str').astype('int').astype('float')
comm_scale = (nc - np.mean(nc)) / np.std(nc)

In [106]:
cont_scale = (data.contrast  - np.mean(data.contrast))/np.std(data.contrast)
nfavs = data.n_favs.astype('str').astype('int').astype('float')
nviews = data.n_views.astype('str').astype('int').astype('float')
y = nfavs / nviews

In [ ]:
data['brightness_scale']=bright_scale
data['contrast_scale'] = cont_scale
data['ntags_scale']=ntags_scale
data['comments_scale'] = comm_scale

In [109]:
data.to_sql('image_stats_del', engine, if_exists='replace')

In [111]:
from sklearn.linear_model import LinearRegression

In [173]:
data['result_y'] = y
data.to_sql('image_stats_del', engine, if_exists='replace')

In [411]:
query = "SELECT * from image_stats_del" #% patient
qr=pd.read_sql_query(query,con)
qr.head()

,level_0,index,picture_id,user_id,static_file,date_posted,n_favs,n_views,n_comments,tag_raw,...,longitude,hexcolors,brightness,contrast,ntags,brightness_scale,contrast_scale,ntags_scale,comments_scale,result_y
0,0,0,6676961391,7891351@N05,https://farm8.staticflickr.com/7146/6676961391...,1326255301,0,232,2,{coffee},...,-73.959667,"{#8d8457,#ddd7b3,#21170b,#a39969,#f6f3db,#b7ad...",169.645785,231,1,1.377681,0.760153,-1.098296,-0.194511,0.000000
1,1,1,15444712858,81650523@N04,https://farm6.staticflickr.com/5608/1544471285...,1414320368,11,4122,1,"{Coffee,Black,White,Cup,Mug,Mugg,Kaffe,Kaffemu...",...,14.860422,"{#37392e,#d0c8ad,#6f6e57,#1f221b,#ece5cb,#928d...",131.139414,217,14,0.125026,0.498208,0.110684,-0.273688,0.002669
2,2,2,15004931544,32157516@N05,https://farm4.staticflickr.com/3953/1500493154...,1414264980,23,2184,5,"{davide978,milano,milan,sempione,colazione,bre...",...,9.172918,"{#b7bbbd,#4e4638,#363328,#d8d9d7,#7f7e75,#9c9e...",140.740993,197,25,0.437376,0.124001,1.133667,0.043020,0.010531
3,3,3,15600916982,56237114@N08,https://farm6.staticflickr.com/5608/1560091698...,1413982237,9,1818,1,"{Rijeka,hrvatska,croatia,summer,panorama,canon...",...,14.455518,"{#8b8b85,#22262c,#bee2f1,#11110e,#a8acab,#f7f1...",167.636340,240,20,1.312312,0.928546,0.668675,-0.273688,0.004950
4,4,4,15407506400,126314760@N06,https://farm6.staticflickr.com/5612/1540750640...,1413908972,1,180,0,"{travel,coffee,cafe,kaffe,relax,terrace}",...,-3.708572,"{#1b120e,#dad7d4,#83776a,#3d2e26,#ae9883,#9f6e...",134.482411,208,6,0.233778,0.329815,-0.633304,-0.352865,0.005556


In [418]:
x1 = qr.brightness_scale; x2 = qr.contrast_scale; x3 = qr.ntags_scale; x4 = qr.comments_scale;
x =np.column_stack((x1,x2,x3,x4))
y = qr.result_y

In [419]:
xlearn = x[0:3000,:]
ylearn = y[0:3000]
xval = x[3000:,:]
yval = y[3000:]

In [422]:
clf = LinearRegression()
clf.fit (xlearn, ylearn)
print clf.coef_
print clf.intercept_

[  7.79905049e-05  -2.30374849e-04  -7.23382833e-04   2.20559201e-03]
0.00456432325076


In [ ]:
ypred = clf.predict(xval)

In [ ]:
a = [-0.41438529,  1.19049025,  0.76167307, -0.27368794]
sum(a*clf.coef_)+clf.intercept_

In [435]:
ypred

array([ 0.00440183,  0.00449345,  0.00441748,  0.00435785,  0.00453112,
        0.00485915,  0.00445823,  0.0042869 ,  0.00467032,  0.0044301 ,
        0.00455855,  0.00425233,  0.00445914,  0.0044728 ,  0.00456223,
        0.00444578,  0.00436329,  0.00480942,  0.00457799,  0.00444047,
        0.00442438,  0.00467466,  0.00447309,  0.00454939,  0.00449626,
        0.00446939,  0.00457515,  0.00458839,  0.00435624,  0.00442516,
        0.00456754,  0.00454727,  0.00440829,  0.00450634,  0.0045394 ,
        0.00466153,  0.00431427,  0.00462768,  0.00460082,  0.0045619 ,
        0.00467491,  0.00444498,  0.00422662,  0.00456047,  0.00452222,
        0.00425398,  0.00470377,  0.00461782,  0.00449847,  0.00429129,
        0.00477341,  0.00457907,  0.00453414,  0.00460335,  0.00446749,
        0.00440479,  0.0046842 ,  0.00430378,  0.00429582,  0.00445626,
        0.00456521,  0.00457801,  0.00461597,  0.00431677,  0.00465325,
        0.00477113,  0.00430199,  0.00449773,  0.00438832,  0.00

In [427]:
qr[3000:]

,level_0,index,picture_id,user_id,static_file,date_posted,n_favs,n_views,n_comments,tag_raw,...,longitude,hexcolors,brightness,contrast,ntags,brightness_scale,contrast_scale,ntags_scale,comments_scale,result_y
3000,2996,2996,3495479747,7891351@N05,https://farm4.staticflickr.com/3640/3495479747...,1241325761,2,562,1,"{""la colombe"",cafe,""la colombe cafe"",""coffee h...",...,-73.996383,"{#17100c,#908e8c,#674f39,#f4f3e8,#422b1c,#7271...",114.558003,254,21,-0.414385,1.190490,0.761673,-0.273688,0.003559
3001,2997,2997,3495478813,7891351@N05,https://farm4.staticflickr.com/3317/3495478813...,1241325737,4,946,2,"{""la colombe"",cafe,""la colombe cafe"",""coffee h...",...,-73.996383,"{#0a0f03,#7d4c09,#ddcb58,#435908,#6f8815,#a1ba...",84.972226,173,23,-1.376843,-0.325047,0.947670,-0.194511,0.004228
3002,2998,2998,3492759380,91271949@N00,https://farm4.staticflickr.com/3638/3492759380...,1241219007,0,234,10,"{koffie,appeltaart,slagroom,koffiekopje,tiktak...",...,5.107398,"{#d2bfa6,#0e0a06,#877766,#4a3c2d,#a99885,#f5e4...",123.545239,224,10,-0.122021,0.629180,-0.261310,0.438905,0.000000
3003,2999,2999,3485350117,36478297@N02,https://farm4.staticflickr.com/3301/3485350117...,1241010749,0,38,0,"{coffee,cappuccino}",...,32.844915,"{#311e1c,#888b8a,#aebbc4,#5d5b58,#b96d40,#1a0a...",118.077151,198,2,-0.299904,0.142712,-1.005298,-0.352865,0.000000
3004,3000,3000,3482253714,48085290@N00,https://farm4.staticflickr.com/3557/3482253714...,1240888174,0,107,1,"{toronto,""ward's island"",""the rectory cafe"",mo...",...,-79.356415,"{#a7ad96,#314930,#d8e9ed,#103122,#5d6043,#b4cb...",157.768026,209,5,0.991285,0.348525,-0.726302,-0.273688,0.000000
3005,3001,3001,3479857835,9942688@N04,https://farm4.staticflickr.com/3375/3479857835...,1240851499,1,460,3,"{poesias,poesia,photo,petricelli,lmax,brazil,a...",...,-46.648120,"{#997a61,#988985,#bca89c,#b59375,#949dbc,#c1c1...",148.364134,91,13,0.685366,-1.859294,0.017685,-0.115334,0.002174
3006,3002,3002,3472077944,85265114@N00,https://farm4.staticflickr.com/3545/3472077944...,1240611293,0,200,6,"{365,project,coffee,cereal,soup,milk,café,addi...",...,-71.259330,"{#100805,#664935,#99907a,#372517,#8d6113,#e1d8...",87.042337,144,11,-1.309500,-0.867646,-0.168312,0.122197,0.000000
3007,3003,3003,3471016215,36900230@N05,https://farm4.staticflickr.com/3374/3471016215...,1240603681,1,137,2,"{coffee,shots,starbucks,brewing,espresso}",...,-6.239697,"{#525e2a,#c9df96,#1e1903,#748157,#9faf70,#3739...",117.135294,240,5,-0.330543,0.928546,-0.726302,-0.194511,0.007299
3008,3004,3004,3467248584,34525698@N00,https://farm4.staticflickr.com/3517/3467248584...,1240447895,4,269,2,"{""NEW BERN"",NC,""PORT CITY JAVA"",""COFFEE HOUSE""...",...,-77.039480,"{#736d60,#601814,#9a9387,#2e2619,#868072,#c7c2...",118.760342,121,14,-0.277679,-1.297984,0.110684,-0.194511,0.014870
3009,3005,3005,3466185743,85265114@N00,https://farm4.staticflickr.com/3578/3466185743...,1240440302,4,560,14,"{365,coffee,addict,espresso,injection,vein,mac...",...,-71.259330,"{#181210,#af9c88,#f5f8f7,#665853,#997669,#3330...",130.150963,246,13,0.092871,1.040808,0.017685,0.755612,0.007143


In [208]:
newdf = pd.DataFrame()
newdf['picture_id'] = data.picture_id[3000:]
newdf['ypredict'] = ypred
newdf['yactual'] = yval

In [210]:
newdf.to_sql('valset_865', engine, if_exists='replace')

In [359]:
query = "SELECT * from image_stats_del" #% patient
query_results=pd.read_sql_query(query,engine)

In [360]:
a =query_results.loc[query_results['picture_id'].isin(['15159853973','2752376454','2813490117'])]

In [328]:
contrast = np.asarray(a.contrast_scale)
bright = np.asarray(a.brightness_scale)
ntags = np.asarray(a.ntags_scale)
ncomments = np.asarray(a.comments_scale)
x = np.column_stack((bright,contrast,ntags,ncomments));

array([[-0.371169  , -0.39988796, -0.81930083, -0.35286488],
       [ 0.38611964,  1.19049025, -0.81930083, -0.03615713],
       [ 0.40317285, -0.92377725,  0.38967921, -0.03615713]])

In [329]:
coef = [8.60414170e-05, -2.38880084e-04, -7.24585265e-04, 2.20345206e-03]
np.shape(x),np.shape(coef)

((3, 4), (4,))

In [368]:
r =np.dot(x,coef) 
r + intercept

array([ 0.0044456 ,  0.0048287 ,  0.00445921])

In [358]:
intercept = 0.00456587911375
-0.00012028 + 0.00456587911375
0.00026282 + 0.00456587911375

0.0048286991137499995

In [348]:
aa =df.loc[df['picture_id'].isin(['15159853973','2752376454','2813490117'])]

In [349]:
aa

,picture_id,ypredict,yactual
3152,2813490117,0.065256,0.049873
3242,2752376454,0.001134,0.000000
3851,15159853973,0.004459,0.004228


In [ ]:
dbname = 'coffee_detai1'
username = 'emclinden'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url

query = "SELECT * from image_stats_del" #% patient
query_results=pd.read_sql_query(query,engine)

In [380]:
import psycopg2

user = 'emclinden' #add your username here (same as previous postgreSQL)            
host = 'localhost'
dbname = 'coffee_detai1'
db = create_engine('postgres://%s%s/%s'%(user,host,dbname))
con = None
con = psycopg2.connect(database = dbname, user = user)

In [384]:
query = "SELECT * from image_stats_del" #% patient
qr=pd.read_sql_query(query,con)

In [386]:
qr

,level_0,index,picture_id,user_id,static_file,date_posted,n_favs,n_views,n_comments,tag_raw,...,longitude,hexcolors,brightness,contrast,ntags,brightness_scale,contrast_scale,ntags_scale,comments_scale,result_y
0,0,0,6676961391,7891351@N05,https://farm8.staticflickr.com/7146/6676961391...,1326255301,0,232,2,{coffee},...,-73.959667,"{#8d8457,#ddd7b3,#21170b,#a39969,#f6f3db,#b7ad...",169.645785,231,1,1.377681,0.760153,-1.098296,-0.194511,0.000000
1,1,1,15444712858,81650523@N04,https://farm6.staticflickr.com/5608/1544471285...,1414320368,11,4122,1,"{Coffee,Black,White,Cup,Mug,Mugg,Kaffe,Kaffemu...",...,14.860422,"{#37392e,#d0c8ad,#6f6e57,#1f221b,#ece5cb,#928d...",131.139414,217,14,0.125026,0.498208,0.110684,-0.273688,0.002669
2,2,2,15004931544,32157516@N05,https://farm4.staticflickr.com/3953/1500493154...,1414264980,23,2184,5,"{davide978,milano,milan,sempione,colazione,bre...",...,9.172918,"{#b7bbbd,#4e4638,#363328,#d8d9d7,#7f7e75,#9c9e...",140.740993,197,25,0.437376,0.124001,1.133667,0.043020,0.010531
3,3,3,15600916982,56237114@N08,https://farm6.staticflickr.com/5608/1560091698...,1413982237,9,1818,1,"{Rijeka,hrvatska,croatia,summer,panorama,canon...",...,14.455518,"{#8b8b85,#22262c,#bee2f1,#11110e,#a8acab,#f7f1...",167.636340,240,20,1.312312,0.928546,0.668675,-0.273688,0.004950
4,4,4,15407506400,126314760@N06,https://farm6.staticflickr.com/5612/1540750640...,1413908972,1,180,0,"{travel,coffee,cafe,kaffe,relax,terrace}",...,-3.708572,"{#1b120e,#dad7d4,#83776a,#3d2e26,#ae9883,#9f6e...",134.482411,208,6,0.233778,0.329815,-0.633304,-0.352865,0.005556
5,5,5,15397619317,8096730@N04,https://farm6.staticflickr.com/5606/1539761931...,1413816439,2,1173,0,"{Oregon,coffee,""Olympus OM-D E-M5"",E-M5,""Stump...",...,-122.681592,"{#0b0907,#908d7b,#9e5e1f,#f9d385,#6d6a5b,#422c...",96.351542,185,10,-1.006662,-0.100523,-0.261310,-0.352865,0.001705
6,6,6,15186661968,69861074@N00,https://farm4.staticflickr.com/3850/1518666196...,1411854385,19,1199,5,"{lifestyle,cafe,DCist,Dolcezza,Mosaic,gibraltar}",...,-77.231278,"{#050505,#afafaf,#747474,#fbfbfb,#323232,#8d8d...",142.557619,254,6,0.496473,1.190490,-0.633304,0.043020,0.015847
7,7,7,15138977569,82684724@N00,https://farm4.staticflickr.com/3869/1513897756...,1411494024,17,2494,0,"{""Scout Coffee ~ San Luis Obispo, California"",...",...,-120.663352,"{#331d11,#e6c0a6,#b0433f,#c28e72,#e3e5e8,#8729...",151.005344,211,10,0.771287,0.385946,-0.261310,-0.352865,0.006816
8,8,8,15040731800,38501716@N08,https://farm4.staticflickr.com/3878/1504073180...,1410633692,53,3438,11,"{snack,casa,brown,dessert,niftyfifty,editadaco...",...,-99.133168,"{#775644,#3c1b0e,#bfb3a5,#5e3926,#a99585,#d6cf...",112.586799,207,47,-0.478511,0.311105,3.179633,0.518081,0.015416
9,9,9,15154582916,100016856@N08,https://farm6.staticflickr.com/5578/1515458291...,1410231364,136,5405,36,"{lunchbreak,break,breaking,icedcoffee,tea,frie...",...,135.768912,"{#edc3a8,#654047,#191226,#a57f67,#9a5040,#c49f...",136.117523,193,20,0.286970,0.049160,0.668675,2.497505,0.025162
